In [0]:
!nvidia-smi

Tue Dec 17 17:59:37 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.44       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P0    32W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
import sys
sys.path.append('/content/drive/My Drive/Luan Van 2019/Tumor/model/src')
from util.npy_generator import *
from util.metrics import *

Using TensorFlow backend.


In [0]:
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, CSVLogger, TensorBoard
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.metrics import binary_accuracy
from model import  cnn
input_size = (64, 64, 64)
batch_size = 4
#CNN_weights_path = "drive/My Drive/Liver3D/pretrained_weights/CNN_all_final.hdf5"
model = cnn.Unet3D(size=(64, 64, 64),base_n_filter = 16, output_activation_name='sigmoid',CNN_weights_path="/content/drive/My Drive/Luan Van 2019/Tumor/weights/weights_model14/best_valid_model14_t2.hdf5")

In [0]:
import tensorflow as tf
def Dice_loss(y_true,y_pred):
    return 1-DICE(y_true,y_pred)
def DICE(y_true, y_pred, smooth=1.0):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

In [0]:
n_epochs = 50
learning_rate = 1e-5
early_stopping_patience = n_epochs-1
steps_per_epoch = 50*240//batch_size
validation_steps = 12*240//batch_size
checkpoint_best_lost_path = "/content/drive/My Drive/Luan Van 2019/Tumor/weights/weight_unet/best_loss_unet_ver2.hdf5"
checkpoint_best_valloss_path = "/content/drive/My Drive/Luan Van 2019/Tumor/weights/weight_unet/best_valid_unet_ver2.hdf5"
checkpoint_best_last_path = "/content/drive/My Drive/Luan Van 2019/Tumor/weights/weight_unet/last_weight_unet_ver2.hdf5"
print('steps_per_epoch: ', steps_per_epoch)
print('validation_steps: ',validation_steps)
#checkpoint_path_Unet3D = "/content/drive/My Drive/Luan Van 2019/Tumor/weights/weight_unet/last_weight_unet_ver2.hdf5"
#model.load_weights(checkpoint_path_Unet3D)
model.compile(optimizer=Adam(lr=learning_rate), loss=Dice_loss, metrics=[VOE ,DICE,binary_accuracy])

steps_per_epoch:  3000
validation_steps:  720


In [0]:
import glob
X_train = glob.glob(r'/content/drive/My Drive/Dataset2/data/data_5/train/scan*')
y_train = glob.glob(r'/content/drive/My Drive/Dataset2/data/data_5/train/gt*')
X_val = glob.glob(r'/content/drive/My Drive/Dataset2/data/data_5/valid/scan*')
y_val = glob.glob(r'/content/drive/My Drive/Dataset2/data/data_5/valid/gt*')
X_train.sort()
y_train.sort()
X_val.sort()
y_val.sort()
X_train=X_train[0:50]
y_train=y_train[0:50]
print(X_train)
print(y_train)
print(X_val)
print(y_val)

['/content/drive/My Drive/Dataset2/data/data_5/train/scan-n0240-s64_64_64.part001.npy', '/content/drive/My Drive/Dataset2/data/data_5/train/scan-n0240-s64_64_64.part002.npy', '/content/drive/My Drive/Dataset2/data/data_5/train/scan-n0240-s64_64_64.part003.npy', '/content/drive/My Drive/Dataset2/data/data_5/train/scan-n0240-s64_64_64.part004.npy', '/content/drive/My Drive/Dataset2/data/data_5/train/scan-n0240-s64_64_64.part005.npy', '/content/drive/My Drive/Dataset2/data/data_5/train/scan-n0240-s64_64_64.part006.npy', '/content/drive/My Drive/Dataset2/data/data_5/train/scan-n0240-s64_64_64.part007.npy', '/content/drive/My Drive/Dataset2/data/data_5/train/scan-n0240-s64_64_64.part008.npy', '/content/drive/My Drive/Dataset2/data/data_5/train/scan-n0240-s64_64_64.part009.npy', '/content/drive/My Drive/Dataset2/data/data_5/train/scan-n0240-s64_64_64.part010.npy', '/content/drive/My Drive/Dataset2/data/data_5/train/scan-n0240-s64_64_64.part011.npy', '/content/drive/My Drive/Dataset2/data/dat

In [0]:
training_generator = npy_generator(X_train,y_train, use_normalize = False, batch_size=batch_size, n_outputs = 1, subtract_mean = None,reshape = True)
validation_generator = npy_generator(X_val,y_val, use_normalize = False, batch_size=batch_size, n_outputs = 1, subtract_mean = None,reshape = True)
model_checkpoint_loss = ModelCheckpoint(checkpoint_best_lost_path, monitor='loss', verbose = 1, save_best_only=True)
model_checkpoint_valloss = ModelCheckpoint(checkpoint_best_valloss_path, monitor='val_loss', verbose = 1, save_best_only=True)
model_checkpoint_last = ModelCheckpoint(checkpoint_best_last_path, verbose = 1, save_best_only=False)
learning_rate_plateau_loss = ReduceLROnPlateau(monitor='loss', factor=0.1, patience=n_epochs//3, verbose=0, mode='auto', min_delta=0.0001, cooldown=0, min_lr=1e-9)
learning_rate_plateau_valloss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=n_epochs//2, verbose=0, mode='auto', min_delta=0.0001, cooldown=0, min_lr=1e-9)
early_stopping = EarlyStopping(verbose = 1, patience=early_stopping_patience)
logfile = "/content/drive/My Drive/Luan Van 2019/Tumor/log/log_model_unet_ver2.csv"
csv_logger = CSVLogger(logfile, separator=',', append=True)
call_backs = [model_checkpoint_loss, model_checkpoint_valloss,model_checkpoint_last ,learning_rate_plateau_loss,csv_logger,learning_rate_plateau_valloss, early_stopping]

In [0]:
model.fit_generator(generator=training_generator,
                        steps_per_epoch=steps_per_epoch,
                        epochs=n_epochs,
                        validation_data=validation_generator,
                        validation_steps=validation_steps,
                        callbacks=call_backs)

Epoch 1/50
 720/3000 [======>.......................] - ETA: 9:02 - loss: 0.4999 - VOE: 0.6169 - DICE: 0.5001 - binary_accuracy: 0.9746
Epoch 00001: loss improved from inf to 0.41548, saving model to /content/drive/My Drive/Luan Van 2019/Tumor/weights/weight_unet/best_loss_unet_ver2.hdf5

Epoch 00001: val_loss improved from inf to 0.49992, saving model to /content/drive/My Drive/Luan Van 2019/Tumor/weights/weight_unet/best_valid_unet_ver2.hdf5

Epoch 00001: saving model to /content/drive/My Drive/Luan Van 2019/Tumor/weights/weight_unet/last_weight_unet_ver2.hdf5
3000/3000 [==============================] - 1117s 372ms/step - loss: 0.4155 - VOE: 0.5368 - DICE: 0.5845 - binary_accuracy: 0.9547 - val_loss: 0.4999 - val_VOE: 0.6169 - val_DICE: 0.5001 - val_binary_accuracy: 0.9746
Epoch 2/50
 720/3000 [======>.......................] - ETA: 2:49 - loss: 0.4881 - VOE: 0.6071 - DICE: 0.5119 - binary_accuracy: 0.9744
Epoch 00002: loss improved from 0.41548 to 0.32743, saving model to /content/